### Basics of RL

<font color='#ed005a'>Value Iteraion </font>: Train a value function to learn which state is more valuable and use this value function to take the action that leads to it. We compute the state value[V(s)] or the Action value function [Q(a,s) / A(a,s)] for this purpose

<font color='#ed005a'>Policy Iteraion </font>: Train the policy directly to learn which action to take given a state. Mostly used in Deep reinforcement learning

<img src='Image/RL_Basics.png' width="700">

Different ways of solving deterministic and finite environments

<img src='Image/RL_algorithms.png' width="700">

<img src=https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/two-types.jpg width="700">

### Model free RL

Since we do not know about the environment dynamics, we try to estimate the Q and V values of different states by trial and errors.

#### <font color='#0175c2'>Monte Carlo Learning</font>

This method of updating value function computes an <font color='#00ba47'>entire episode</font> to get a trajectory using the old policy and then updates using the total return $G_{t}$

$$V(s_k) = E(r_k + \gamma V(s_{k+1})) $$

We estimate the V value by simple update term as follows:

$$G_{t} = \sum_{k=1}^{n}\gamma^{k}r_k$$
$$V^{new}(s_k) = V^{old}(s_k) + \frac{1}{n} (G_{t} - V^{old}(s_k))$$
$$Q^{new}(s_k,a_k) = Q^{old}(s_k,a_k) + \frac{1}{n} (G_{t} - Q^{old}(s_k,a_k))$$

If we reach the optimal value of $V(s_k)$, the difference term $(R_{\sum} - V^{old}(s_k)$ becomes 0 and $V(s_k)$ stops updating. But this gives equal weightage to all the steps hence, this is actually slow and we use _Temporal Difference_ which gives weightage to results nearer to the rewards. 



In [1]:
import numpy as np
import sys
import gym
import random
from collections import defaultdict, deque
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
env = gym.make('Blackjack-v1')

def generate_episode_from_limit_stochastic(bj_env):
    episode = []
    state = bj_env.reset()
    while True:
        probs = [0.8, 0.2] if state[0] > 18 else [0.2, 0.8]
        action = np.random.choice(np.arange(2), p=probs)
        next_state, reward, done, info = bj_env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

def mc_prediction_q(env, num_episodes, generate_episode, gamma=1.0):
    # initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        # generate an episode
        episode = generate_episode(env)
        # obtain the states, actions, and rewards
        states, actions, rewards = zip(*episode)
        # prepare for discounting
        discounts = np.array([gamma**i for i in range(len(rewards)+1)])
        # update the sum of the returns, number of visits, and action-value 
        # function estimates for each state-action pair in the episode
        for i, state in enumerate(states):
            returns_sum[state][actions[i]] += sum(rewards[i:]*discounts[:-(1+i)])
            N[state][actions[i]] += 1.0
            Q[state][actions[i]] = returns_sum[state][actions[i]] / N[state][actions[i]]
    return Q

#### <font color='#0175c2'>Temporal Difference</font>

This method estimates the value function at <font color='#00ba47'>each step in an episode</font> by using the value function of the next state.

$$V^{new}(s_k) = V^{old}(s_k) + \alpha (\underbrace{\color{#ed005a}{r_k + \gamma V^{old}(s_{k+1})}}_{\color{#0175c2}{G_t}} - V^{old}(s_k))$$

$G_{t}$ : TD Target Estimate

This is kind of 1 $\Delta t$ delay between an action and reward and things correlated with 1 $\Delta t$ will get strengthened in this TD(0) framework. 

$$V(s_k) = E(r_k + \gamma r_{k+1} + \gamma^2 V(s_{k+1})) $$
Therefore TD(1),
$$V^{new}(s_k) = V^{old}(s_k) + \alpha (r_k + \gamma r_{k+1} + \gamma^2 V^{old}(s_{k+1}) - V^{old}(s_k))$$
Here we are considering 2 steps into the future. This is called <font color='#0175c2'> n step Bootstrapping</font>. This way we can go on to generate TD(N) famework. If n $\to$ $\infty$, this will converge to Monte Carlo Learning.

##### <font color='#00ba47'>SARSA: _on-policy TD control method_ </font>

State-Action-Reward-State-Action
$$Q^{new}(s_k,a_k) = Q^{old}(s_k,a_k) + \alpha (r_k + \gamma Q^{old}(s_{k+1},a_{k+1}) - Q^{old}(s_k,a_k))$$

Sarsa is generally safer as it always follows what it thinks is best. Eg teaching a teenager to drive, here random actions can lead to accidents and hence Sarsa is preferred.

In [3]:
env = gym.make('CliffWalking-v0')
def update_Q_sarsa(alpha, gamma, Q, state, action, reward, next_state=None, next_action=None):
    """Returns updated Q-value for the most recent experience."""
    current = Q[state][action]  # estimate in Q-table (for current state, action pair)
    # get value of state, action pair at next time step
    Qsa_next = Q[next_state][next_action] if next_state is not None else 0    
    target = reward + (gamma * Qsa_next)               # construct TD target
    new_value = current + (alpha * (target - current)) # get updated value
    return new_value

def epsilon_greedy(Q, state, nA, eps):
    if random.random() > eps: # select greedy action with probability epsilon
        return np.argmax(Q[state])
    else:                     # otherwise, select an action randomly
        return random.choice(np.arange(nA))

In [4]:
def sarsa(env, num_episodes, alpha, gamma=1.0, plot_every=100):
    nA = env.action_space.n                # number of actions
    Q = defaultdict(lambda: np.zeros(nA))  # initialize empty dictionary of arrays
    
    # monitor performance
    tmp_scores = deque(maxlen=plot_every)     # deque for keeping track of scores
    avg_scores = deque(maxlen=num_episodes)   # average scores over every plot_every episodes
    
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 100 == 0:
            print("\rEpisode {}/{}".format(i_episode, num_episodes), end="")
            sys.stdout.flush()   
        score = 0                                             # initialize score
        state = env.reset()                                   # start episode
        
        eps = 1.0 / i_episode                                 # set value of epsilon
        action = epsilon_greedy(Q, state, nA, eps)            # epsilon-greedy action selection
        
        while True:
            next_state, reward, done, info = env.step(action) # take action A, observe R, S'
            score += reward                                   # add reward to agent's score
            if not done:
                next_action = epsilon_greedy(Q, next_state, nA, eps) # epsilon-greedy action
                Q[state][action] = update_Q_sarsa(alpha, gamma, Q, \
                                                  state, action, reward, next_state, next_action)
                
                state = next_state     # S <- S'
                action = next_action   # A <- A'
            if done:
                Q[state][action] = update_Q_sarsa(alpha, gamma, Q, \
                                                  state, action, reward)
                tmp_scores.append(score)    # append score
                break
        if (i_episode % plot_every == 0):
            avg_scores.append(np.mean(tmp_scores))
    # plot performance
    plt.plot(np.linspace(0,num_episodes,len(avg_scores),endpoint=False), np.asarray(avg_scores))
    plt.xlabel('Episode Number')
    plt.ylabel('Average Reward (Over Next %d Episodes)' % plot_every)
    plt.show()
    # print best 100-episode performance
    print(('Best Average Reward over %d Episodes: ' % plot_every), np.max(avg_scores))    
    return Q

##### <font color='#00ba47'> Sarsamax/Q learning: _off-policy TD control method_ </font>

Q learning is just Temporal Difference Learning but on Q Function

$$Q^{new}(s_k,a_k) = Q^{old}(s_k,a_k) + \alpha (r_{k+1} + \gamma {\color{#ed005a}{\mathop{max}_{\textbf{a} \in A} Q^{old}(s_{k+1},a)}} - Q^{old}(s_k,a_k))$$

We can compute any random action to get $r_{k=1}$ but while computing $Q^{new}(s_k,a_k)$, we maximize over *a*. Here is the updated line in the above code.

`a_ = np.argmax([Q[(s_, a)] for a in range(env.action_space.n)])`

Q Learning is faster and can learn from imitation and experience replay. \
Different strategies exist for introducing this randomness : epsilon greedy, softmax exploration, upper confidence bound, thompson sampling etc

In [5]:
def update_Q_sarsamax(alpha, gamma, Q, state, action, reward, next_state=None):
    """Returns updated Q-value for the most recent experience."""
    current = Q[state][action]  # estimate in Q-table (for current state, action pair)
    Qsa_next = np.max(Q[next_state]) if next_state is not None else 0  # value of next state 
    target = reward + (gamma * Qsa_next)               # construct TD target
    new_value = current + (alpha * (target - current)) # get updated value 
    return new_value

## rest function ie sarsa remains same just change update_Q_sarsa --> update_Q_sarsamax

##### <font color='#00ba47'> Expected Sarsa: _on-policy TD control method_ </font>

$$Q^{new}(s_k,a_k) = Q^{old}(s_k,a_k) + \alpha (r_{k+1} + \gamma {\color{#ed005a}{\sum_{\textbf{a}} \pi (a|s_{t+1}) Q^{old}(s_{k+1},a)}} - Q^{old}(s_k,a_k))$$

In [6]:
def update_Q_expsarsa(alpha, gamma, nA, eps, Q, state, action, reward, next_state=None):
    """Returns updated Q-value for the most recent experience."""
    current = Q[state][action]         # estimate in Q-table (for current state, action pair)
    policy_s = np.ones(nA) * eps / nA  # current policy (for next state S')
    policy_s[np.argmax(Q[next_state])] = 1 - eps + (eps / nA) # greedy action
    Qsa_next = np.dot(Q[next_state], policy_s)         # get value of state at next time step
    target = reward + (gamma * Qsa_next)               # construct target
    new_value = current + (alpha * (target - current)) # get updated value 
    return new_value